# Bot-E Image Meme Generation

This notebook currently focuses on creating memes for bot-e questions.

# 

In [29]:
from PIL import Image, ImageDraw, ImageFont
import requests
from io import BytesIO

def hex_to_rgb(value):
    value = value.lstrip('#')
    length = len(value)
    return tuple(
        int(value[i:i + length // 3], 16) for i in range(0, length, length // 3)
    )
    
def create_meme(image_path, font_path, text1, text2=None, position='bottom', alignment='right', font_size=40, font_color='#FFFFFF', horizontal_margin=10, vertical_margin=10):
    img = Image.open(image_path)
    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype(font_path, font_size)

    text_width1, text_height1 = draw.textsize(text1, font=font)
    image_width, image_height = img.size

    text2_width = text2_height = 0
    if text2:
        text2_width, text2_height = draw.textsize(text2, font=font)

    # Determine text position based on the specified position and alignment
    x1 = x2 = 0
    y1 = y2 = 0
    line_spacing = 5  # adjust line spacing to your preference

    if position == 'top':
        y1 = vertical_margin
        if text2:
            y2 = y1 + text_height1 + line_spacing
    elif position == 'bottom':
        y1 = image_height - text_height1 - vertical_margin
        if text2:
            y1 = y1 - text2_height - line_spacing  # adjust y1 upwards to fit second line
            y2 = image_height - text2_height - vertical_margin

    if alignment == 'left':
        x1 = x2 = horizontal_margin
    elif alignment == 'right':
        x1 = x2 = image_width - max(text_width1, text2_width) - horizontal_margin

    draw.text((x1, y1), text1, hex_to_rgb(font_color), font=font)
    if text2:
        draw.text((x2, y2), text2, hex_to_rgb(font_color), font=font)

    img.save('meme.png')
    img.show()

# Example Usage
create_meme(
    image_path='images/memes/7KfmXdL3Kng.png',
    #image_path='images/memes/7OGx5myhvMn.png',
    font_path='fonts/Lato-Regular.ttf',
    text1='Squash',
    text2='Bros',
    position='top',
    alignment='left',
    font_size=20,
    font_color='#ffffff',
    horizontal_margin=20,
    vertical_margin=270
)

/var/folders/dv/ts7drh5n0hb48yx2_5vpm3kw0000gq/T/ipykernel_64553/2035832574.py:17: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width1, text_height1 = draw.textsize(text1, font=font)
/var/folders/dv/ts7drh5n0hb48yx2_5vpm3kw0000gq/T/ipykernel_64553/2035832574.py:22: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text2_width, text2_height = draw.textsize(text2, font=font)
